In [1]:
# -----------------------------------------------
# 🌐 TransFlip App with Splash Animation & Dark Mode
# -----------------------------------------------

import tkinter as tk
from PIL import Image, ImageTk
import time
from tkinter import ttk

# -----------------------------------------------
# 🖼️ Splash Screen with Fade-In, Typing Text, and Loader
# -----------------------------------------------
def show_splash():
    splash = tk.Tk()
    splash.title("Welcome")
    splash.geometry("1000x650")
    splash.configure(bg="#ffffff")
    splash.overrideredirect(True)

    # Load splash image
    try:
        img = Image.open("translator_logo.jpg")
        img = img.resize((1000, 650))
        img_tk = ImageTk.PhotoImage(img)
        label = tk.Label(splash, image=img_tk)
        label.image = img_tk
        label.place(x=0, y=0)
    except:
        label = tk.Label(splash, text="🌐 TransFlip App", font=("Helvetica", 30), bg="#ffffff")
        label.pack(expand=True)

    # Typing animation label
    typing_label = tk.Label(splash, text="", font=("Helvetica", 20, "italic"), bg="#ffffff", fg="#1a1a1a")
    typing_label.place(relx=0.5, rely=0.85, anchor="center")

    def type_text(text, i=0):
        if i < len(text):
            typing_label.config(text=text[:i+1])
            splash.after(100, lambda: type_text(text, i+1))

    type_text("Starting Real TransFlip App...")

    # Add loader bar
    progress = ttk.Progressbar(splash, orient="horizontal", length=400, mode="determinate")
    progress.place(relx=0.5, rely=0.93, anchor="center")

    def load_bar(value=0):
        if value <= 100:
            progress["value"] = value
            splash.after(30, lambda: load_bar(value + 2))
        else:
            splash.destroy()
            run_app()

    splash.attributes("-alpha", 0.0)

    def fade_in(alpha=0.0):
        if alpha < 1.0:
            splash.attributes("-alpha", alpha)
            splash.after(50, lambda: fade_in(alpha + 0.05))
        else:
            load_bar()

    fade_in()
    splash.mainloop()

# -----------------------------------------------
# 🌐 TransFlip App Window
# -----------------------------------------------
def run_app():
    import tkinter as tk
    from tkinter import ttk
    from deep_translator import GoogleTranslator
    from gtts import gTTS
    from playsound import playsound
    import speech_recognition as sr
    import os

    lang_code_map = {
        "English": "en",
        "Hindi": "hi",
        "Marathi": "mr",
        "Spanish": "es",
        "French": "fr",
        "German": "de",
        "Italian": "it",
        "Chinese (Simplified)": "zh-CN",
        "Japanese": "ja",
        "Korean": "ko",
        "Russian": "ru",
        "Arabic": "ar",
        "Portuguese": "pt",
        "Bengali": "bn",
        "Tamil": "ta",
        "Telugu": "te",
        "Gujarati": "gu",
        "Urdu": "ur",
        "Punjabi": "pa"
    }

    def speech_to_text():
        r = sr.Recognizer()
        with sr.Microphone() as source:
            input_text.delete("1.0", tk.END)
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                input_text.insert(tk.END, text)
            except sr.UnknownValueError:
                input_text.insert(tk.END, "Could not understand")
            except sr.RequestError:
                input_text.insert(tk.END, "Network error")

    def translate_text():
        source_lang = lang_code_map[from_lang.get()]
        target_lang = lang_code_map[to_lang.get()]
        original = input_text.get("1.0", tk.END).strip()

        if not original:
            return

        try:
            translated = GoogleTranslator(source=source_lang, target=target_lang).translate(original)
            output_text.delete("1.0", tk.END)
            output_text.insert(tk.END, translated)
            history_list.insert(tk.END, f"{original} → {translated}")

            if auto_speak.get():
                speak_text(translated, target_lang)

        except Exception as e:
            output_text.insert(tk.END, f"Error: {e}")

    def speak_text(text=None, lang=None):
        text = text or output_text.get("1.0", tk.END).strip()
        lang = lang or lang_code_map[to_lang.get()]
        if text:
            try:
                tts = gTTS(text=text, lang=lang)
                filename = "speak.mp3"
                tts.save(filename)
                playsound(filename)
                os.remove(filename)
            except Exception as e:
                print("Error:", e)

    def clear_all():
        input_text.delete("1.0", tk.END)
        output_text.delete("1.0", tk.END)
        history_list.delete(0, tk.END)

    def on_history_select(event):
        selection = history_list.curselection()
        if selection:
            line = history_list.get(selection[0])
            if "→" in line:
                original = line.split("→")[0].strip()
            else:
                original = line.strip()
            input_text.delete("1.0", tk.END)
            input_text.insert(tk.END, original)

    def toggle_theme():
        if theme_var.get():
            apply_dark_theme()
        else:
            apply_light_theme()

    def apply_light_theme():
        root.configure(bg="#f0faff")
        lang_frame.configure(bg="#f0faff")
        text_frame.configure(bg="#f0faff")
        button_frame.configure(bg="#f0faff")
        history_label.configure(bg="#f0faff", fg="black")

    def apply_dark_theme():
        root.configure(bg="#2e2e2e")
        lang_frame.configure(bg="#2e2e2e")
        text_frame.configure(bg="#2e2e2e")
        button_frame.configure(bg="#2e2e2e")
        history_label.configure(bg="#2e2e2e", fg="white")

    root = tk.Tk()
    root.title("🔄 TransFlip App")
    root.geometry("1000x650")
    root.configure(bg="#f0faff")

    theme_var = tk.BooleanVar(value=False)

    lang_frame = tk.Frame(root, bg="#f0faff")
    lang_frame.pack(pady=15)

    ttk.Label(lang_frame, text="From:").pack(side=tk.LEFT, padx=5)
    from_lang = ttk.Combobox(lang_frame, values=list(lang_code_map.keys()), state="readonly", width=14)
    from_lang.set("English")
    from_lang.pack(side=tk.LEFT, padx=5)

    ttk.Label(lang_frame, text="To:").pack(side=tk.LEFT, padx=5)
    to_lang = ttk.Combobox(lang_frame, values=list(lang_code_map.keys()), state="readonly", width=14)
    to_lang.set("Hindi")
    to_lang.pack(side=tk.LEFT, padx=5)

    auto_speak = tk.BooleanVar()
    auto_check = ttk.Checkbutton(lang_frame, text="🔊 Auto Speak", variable=auto_speak)
    auto_check.pack(side=tk.LEFT, padx=10)

    theme_switch = ttk.Checkbutton(lang_frame, text="🌙 Dark Mode", variable=theme_var, command=toggle_theme)
    theme_switch.pack(side=tk.LEFT, padx=10)

    text_frame = tk.Frame(root, bg="#f0faff")
    text_frame.pack(pady=10)

    input_text = tk.Text(text_frame, width=45, height=10, font=("Helvetica", 12))
    input_text.grid(row=0, column=0, padx=10)

    output_text = tk.Text(text_frame, width=45, height=10, font=("Helvetica", 12), fg="green")
    output_text.grid(row=0, column=1, padx=10)

    button_frame = tk.Frame(root, bg="#f0faff")
    button_frame.pack(pady=15)

    ttk.Button(button_frame, text="🎤 Speak", command=speech_to_text).grid(row=0, column=0, padx=10)
    ttk.Button(button_frame, text="🔁 Translate", command=translate_text).grid(row=0, column=1, padx=10)
    ttk.Button(button_frame, text="🔊 Speak Output", command=speak_text).grid(row=0, column=2, padx=10)
    ttk.Button(button_frame, text="🧹 Clear", command=clear_all).grid(row=0, column=3, padx=10)

    history_label = tk.Label(root, text="🕘 Translation History", font=("Helvetica", 12, "bold"), bg="#f0faff")
    history_label.pack(pady=5)

    history_list = tk.Listbox(root, width=110, height=6, font=("Helvetica", 10))
    history_list.pack()
    history_list.bind("<<ListboxSelect>>", on_history_select)

    apply_light_theme()
    root.mainloop()

# -----------------------------------------------
# 🚀 Launch the App
# -----------------------------------------------
show_splash()


In [2]:
# =================== SMART EXPENSE APP MYSQL VERSION ====================
import tkinter as tk
from tkinter import messagebox
from datetime import datetime
import hashlib
import mysql.connector
from mysql.connector import Error
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# ---------------- MYSQL CONFIG ----------------
MYSQL_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "Teju1234",  # <-- apna MySQL password
    "database": "smart_expense_db"
}

# ---------------- DATABASE FUNCTIONS ----------------
def create_database(db_name="smart_expense
                    _db"):
    try:
        conn = mysql.connector.connect(
            host=MYSQL_CONFIG["host"],
            user=MYSQL_CONFIG["user"],
            password=MYSQL_CONFIG["password"]
        )
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        cursor.close()
        conn.close()
    except Error as e:
        print(f"Error creating database: {e}")

def create_connection():
    try:
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        if conn.is_connected():
            return conn
    except Error as e:
        print(f"Error connecting to database: {e}")
        return None

def create_tables():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS users (
                username VARCHAR(50) PRIMARY KEY,
                password_hash VARCHAR(255),
                created_at DATETIME
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS expenses (
                id INT AUTO_INCREMENT PRIMARY KEY,
                username VARCHAR(50),
                category VARCHAR(100),
                amount FLOAT,
                note TEXT,
                date DATETIME
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS goal (
                username VARCHAR(50) PRIMARY KEY,
                amount FLOAT,
                updated_at DATETIME
            )
        """)
        conn.commit()
        cursor.close()
        conn.close()

# ---------------- HELPER ----------------
def hash_password(password):
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

# ---------------- DATABASE CLASS ----------------
class DB:
    def __init__(self):
        create_database()
        create_tables()
        self.conn = create_connection()
        self.cursor = self.conn.cursor(buffered=True)

    # User functions
    def add_user(self, username, password_plain):
        ph = hash_password(password_plain)
        try:
            self.cursor.execute(
                "INSERT INTO users (username, password_hash, created_at) VALUES (%s, %s, %s)",
                (username, ph, datetime.utcnow())
            )
            self.conn.commit()
            return True
        except mysql.connector.IntegrityError:
            return False

    def verify_user(self, username, password_plain):
        self.cursor.execute("SELECT password_hash FROM users WHERE username=%s", (username,))
        row = self.cursor.fetchone()
        if not row:
            return False
        return row[0] == hash_password(password_plain)

    # Expense functions
    def add_expense(self, username, category, amount, note=''):
        self.cursor.execute(
            "INSERT INTO expenses (username, category, amount, note, date) VALUES (%s, %s, %s, %s, %s)",
            (username, category, float(amount), note, datetime.utcnow())
        )
        self.conn.commit()

    def get_expenses_by_user(self, username):
        self.cursor.execute(
            "SELECT id, category, amount, note, date FROM expenses WHERE username=%s ORDER BY date DESC",
            (username,)
        )
        return self.cursor.fetchall()

    def get_category_summary(self, username):
        self.cursor.execute(
            "SELECT category, SUM(amount) FROM expenses WHERE username=%s GROUP BY category",
            (username,)
        )
        return self.cursor.fetchall()

    def get_total_spent_month(self, username):
        month_prefix = datetime.utcnow().strftime("%Y-%m")
        like_pattern = month_prefix + "%"
        self.cursor.execute(
            "SELECT SUM(amount) FROM expenses WHERE username=%s AND DATE_FORMAT(date, '%%Y-%%m')=%s",
            (username, month_prefix)
        )
        row = self.cursor.fetchone()
        return row[0] if row and row[0] is not None else 0.0

    # Goal functions
    def set_goal(self, username, amount):
        now = datetime.utcnow()
        self.cursor.execute(
            "INSERT INTO goal (username, amount, updated_at) VALUES (%s, %s, %s) "
            "ON DUPLICATE KEY UPDATE amount=%s, updated_at=%s",
            (username, float(amount), now, float(amount), now)
        )
        self.conn.commit()

    def get_goal(self, username):
        self.cursor.execute("SELECT amount FROM goal WHERE username=%s", (username,))
        row = self.cursor.fetchone()
        return row[0] if row else None

# ---------------- GUI: Login / Register ----------------
class LoginWindow:
    def __init__(self, db: DB):
        self.db = db
        if not self.db.verify_user("admin", "admin"):
            self.db.add_user("admin", "admin")
        self.root = tk.Tk()
        self.root.title("Smart Expense - Login")
        self.root.geometry("360x220")
        self.build_ui()

    def build_ui(self):
        frm = tk.Frame(self.root, padx=12, pady=12)
        frm.pack(expand=True, fill=tk.BOTH)
        tk.Label(frm, text="Username:").grid(row=0, column=0, sticky="w")
        self.ent_user = tk.Entry(frm)
        self.ent_user.grid(row=0, column=1, pady=6)
        tk.Label(frm, text="Password:").grid(row=1, column=0, sticky="w")
        self.ent_pass = tk.Entry(frm, show="*")
        self.ent_pass.grid(row=1, column=1, pady=6)
        tk.Button(frm, text="Login", width=12, command=self.login).grid(row=2, column=0, pady=12)
        tk.Button(frm, text="Register", width=12, command=self.register).grid(row=2, column=1, pady=12)
        self.lbl_status = tk.Label(frm, text="", fg="red")
        self.lbl_status.grid(row=3, column=0, columnspan=2)

    def login(self):
        user = self.ent_user.get().strip()
        pw = self.ent_pass.get().strip()
        if not user or not pw:
            messagebox.showwarning("Input needed", "Enter username & password")
            return
        if self.db.verify_user(user, pw):
            self.root.destroy()
            MainApp(self.db, user).run()
        else:
            self.lbl_status.config(text="Login failed: check credentials")

    def register(self):
        user = self.ent_user.get().strip()
        pw = self.ent_pass.get().strip()
        if not user or not pw:
            messagebox.showwarning("Input needed", "Enter username & password")
            return
        if self.db.add_user(user, pw):
            messagebox.showinfo("Registered", "User registered successfully")
        else:
            messagebox.showerror("Register failed", "Username already exists")

    def run(self):
        self.root.mainloop()

# ---------------- Main App ----------------
class MainApp:
    def __init__(self, db: DB, username: str):
        self.db = db
        self.username = username
        self.root = tk.Tk()
        self.root.title(f"Smart Expense - {username}")
        self.root.geometry("720x480")
        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Welcome, {self.username}", font=(None,14,"bold")).pack()
        tk.Button(self.root, text="Add Expense", command=self.add_expense_ui).pack(pady=5)
        tk.Button(self.root, text="Show Report", command=self.show_report).pack(pady=5)
        tk.Button(self.root, text="Set Goal", command=self.set_goal_ui).pack(pady=5)
        self.frm_list = tk.Frame(self.root)
        self.frm_list.pack(fill=tk.BOTH, expand=True)
        self.refresh_expense_list()

    def add_expense_ui(self):
        win = tk.Toplevel(self.root)
        win.title("Add Expense")
        tk.Label(win, text="Category:").grid(row=0, column=0)
        ent_cat = tk.Entry(win)
        ent_cat.grid(row=0, column=1)
        tk.Label(win, text="Amount:").grid(row=1, column=0)
        ent_amt = tk.Entry(win)
        ent_amt.grid(row=1, column=1)
        tk.Label(win, text="Note (optional):").grid(row=2, column=0)
        ent_note = tk.Entry(win)
        ent_note.grid(row=2, column=1)
        def save():
            cat = ent_cat.get().strip()
            amt = ent_amt.get().strip()
            note = ent_note.get().strip()
            if not cat or not amt:
                messagebox.showwarning("Input", "Enter category & amount")
                return
            try:
                amt = float(amt)
            except:
                messagebox.showerror("Invalid", "Amount must be number")
                return
            self.db.add_expense(self.username, cat, amt, note)
            messagebox.showinfo("Added", f"{cat} - ₹{amt}")
            win.destroy()
            self.refresh_expense_list()
        tk.Button(win, text="Add", command=save).grid(row=3,column=0,columnspan=2)

    def set_goal_ui(self):
        win = tk.Toplevel(self.root)
        win.title("Set Monthly Goal")
        tk.Label(win, text="Goal (₹):").grid(row=0,column=0)
        ent = tk.Entry(win)
        ent.grid(row=0,column=1)
        def save():
            val = ent.get().strip()
            if not val:
                messagebox.showwarning("Input","Enter numeric goal")
                return
            try:
                val = float(val)
            except:
                messagebox.showerror("Invalid","Goal must be number")
                return
            self.db.set_goal(self.username,val)
            messagebox.showinfo("Saved", f"Goal set: ₹{val}")
            win.destroy()
        tk.Button(win, text="Save", command=save).grid(row=1,column=0,columnspan=2)

    def show_report(self):
        data = self.db.get_category_summary(self.username)
        if not data:
            messagebox.showinfo("Report","No expenses yet")
            return
        cats = [d[0] for d in data]
        amts = [d[1] for d in data]
        fig, ax = plt.subplots(figsize=(6,6))
        ax.pie(amts, labels=cats, autopct="%1.1f%%")
        ax.set_title("Expenses by Category")
        plt.show()

    def refresh_expense_list(self):
        for w in self.frm_list.winfo_children():
            w.destroy()
        rows = self.db.get_expenses_by_user(self.username)
        if not rows:
            tk.Label(self.frm_list, text="No expenses yet").pack()
            return
        for r in rows:
            tk.Label(self.frm_list, text=f"{r[1]} - ₹{r[2]:.2f} | {r[3]} | {r[4]}").pack()

    def run(self):
        self.root.mainloop()

# ---------------- RUN ----------------
if __name__ == "__main__":
    database = DB()
    login = LoginWindow(database)
    login.run()


SyntaxError: unterminated string literal (detected at line 20) (2882634810.py, line 20)

In [3]:
import sqlite3
from tkinter import *
from tkinter import messagebox, ttk
from datetime import datetime, timedelta

# -------------------- DATABASE SETUP -------------------- #
def create_connection():
    conn = sqlite3.connect("inventory.db")
    return conn

def create_table():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS inventory (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            category TEXT,
            quantity INTEGER,
            expiry_date DATE,
            added_date DATE
        )
    """)
    conn.commit()
    conn.close()

create_table()

# -------------------- INVENTORY OPERATIONS -------------------- #
def add_item(name, category, quantity, expiry_date):
    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO inventory (name, category, quantity, expiry_date, added_date)
            VALUES (?, ?, ?, ?, ?)
        """, (name, category, quantity, expiry_date, datetime.now().date()))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", f"{name} added successfully!")
        view_items()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def update_item(item_id, name, category, quantity, expiry_date):
    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE inventory SET name=?, category=?, quantity=?, expiry_date=?
            WHERE id=?
        """, (name, category, quantity, expiry_date, item_id))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", f"{name} updated successfully!")
        view_items()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def delete_item(item_id):
    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("DELETE FROM inventory WHERE id=?", (item_id,))
        conn.commit()
        conn.close()
        messagebox.showinfo("Deleted", "Item deleted successfully!")
        view_items()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def get_items():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM inventory")
    rows = cursor.fetchall()
    conn.close()
    return rows

def alert_expiry(days=7):
    conn = create_connection()
    cursor = conn.cursor()
    today = datetime.now().date()
    alert_date = today + timedelta(days=days)
    cursor.execute("SELECT name, expiry_date FROM inventory WHERE expiry_date <= ?", (alert_date,))
    items = cursor.fetchall()
    conn.close()
    return items

# -------------------- GUI -------------------- #
root = Tk()
root.title("Smart Inventory with Expiry Tracker")
root.geometry("800x500")

# Labels & Entries
Label(root, text="Item Name:").place(x=20, y=20)
name_entry = Entry(root)
name_entry.place(x=120, y=20)

Label(root, text="Category:").place(x=20, y=50)
category_entry = Entry(root)
category_entry.place(x=120, y=50)

Label(root, text="Quantity:").place(x=20, y=80)
quantity_entry = Entry(root)
quantity_entry.place(x=120, y=80)

Label(root, text="Expiry Date (YYYY-MM-DD):").place(x=20, y=110)
expiry_entry = Entry(root)
expiry_entry.place(x=180, y=110)

selected_item_id = None

# -------------------- TREEVIEW -------------------- #
columns = ("ID", "Name", "Category", "Quantity", "Expiry Date", "Added Date")
tree = ttk.Treeview(root, columns=columns, show="headings")
for col in columns:
    tree.heading(col, text=col)
tree.place(x=20, y=200, width=750, height=250)

def view_items():
    for row in tree.get_children():
        tree.delete(row)
    for item in get_items():
        tree.insert("", END, values=item)

view_items()

def on_tree_select(event):
    global selected_item_id
    selected = tree.focus()
    if selected:
        values = tree.item(selected, "values")
        selected_item_id = values[0]
        name_entry.delete(0, END)
        name_entry.insert(0, values[1])
        category_entry.delete(0, END)
        category_entry.insert(0, values[2])
        quantity_entry.delete(0, END)
        quantity_entry.insert(0, values[3])
        expiry_entry.delete(0, END)
        expiry_entry.insert(0, values[4])

tree.bind("<<TreeviewSelect>>", on_tree_select)

# -------------------- BUTTON FUNCTIONS -------------------- #
def add_button():
    name = name_entry.get()
    category = category_entry.get()
    quantity = quantity_entry.get()
    expiry = expiry_entry.get()
    if name and category and quantity and expiry:
        add_item(name, category, int(quantity), expiry)
    else:
        messagebox.showerror("Error", "Please fill all fields.")

def update_button():
    global selected_item_id
    if selected_item_id:
        update_item(selected_item_id, name_entry.get(), category_entry.get(), int(quantity_entry.get()), expiry_entry.get())
    else:
        messagebox.showerror("Error", "Select an item to update.")

def delete_button_func():
    global selected_item_id
    if selected_item_id:
        delete_item(selected_item_id)
    else:
        messagebox.showerror("Error", "Select an item to delete.")

def expiry_alert_button():
    items = alert_expiry()
    if items:
        msg = "Items near expiry:\n"
        for name, expiry in items:
            msg += f"{name} - {expiry}\n"
        messagebox.showwarning("Expiry Alert", msg)
    else:
        messagebox.showinfo("No Alerts", "No items are near expiry.")

# Buttons
Button(root, text="Add Item", command=add_button, bg="green", fg="white").place(x=20, y=150, width=100)
Button(root, text="Update Item", command=update_button, bg="blue", fg="white").place(x=140, y=150, width=100)
Button(root, text="Delete Item", command=delete_button_func, bg="red", fg="white").place(x=260, y=150, width=100)
Button(root, text="Expiry Alert", command=expiry_alert_button, bg="orange", fg="white").place(x=380, y=150, width=120)

root.mainloop()
